# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS longpb_udacity
    WITH REPLICATION =
    {'class': 'SimpleStrategy', 'replication_factor': 1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('longpb_udacity')
except Exception as e:
    print(e)

### 1. Create tables for query 1:
`Table Name: song_by_session
column 1: Session ID
column 2: Item in Session
column 3: Artist Name
column 4: Song Title
column 5: Song Length
PRIMARY KEY (Session ID, Item in Session)`

**Session ID** and **Item in Session** are the columns that ares used as **composite PRIMARY KEY** of the table. **Session ID** was used as **a partiton key** because the queries will filter by this column. **Item in Session** column was used as **clustering columns** to help make up a unique key.


In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

create_query1 = "CREATE TABLE IF NOT EXISTS song_by_session "
create_query1 = create_query1 + ('(sessionId INT, \
                                   itemInSession INT, \
                                   artist TEXT, \
                                   song TEXT, \
                                   song_length FLOAT, \
                                   PRIMARY KEY (sessionId, itemInSession))')
try:
    session.execute(create_query1)
except Exception as e:
    print(e)                    

#### Insert data into "song_by_session" table

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `insert_query1` variable
        insert_query1 = "INSERT INTO song_by_session (sessionId, \
                                                      itemInSession, \
                                                      artist, \
                                                      song, \
                                                      song_length) "
        insert_query1 = insert_query1 + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insert_query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into "song_by_session" table

In [10]:
## Add in the SELECT statement to verify the data was entered into the table
query1 = "SELECT artist, song, song_length FROM song_by_session \
          WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.song_length)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Create tables for query 2: 
`Table Name: song_by_artist
column 1: User ID
column 2: Session ID
column 3: Item in Session
column 4: Artist Name
column 5: Song Title
column 6: User's first name
column 7: User's last name
PRIMARY KEY (User ID, Session ID, Item In Session)`

**User ID** and **Session ID** are the columns that ares used as **composite partition key** of the table's **PRIMARY KEY** because the queries will filter by these columns. **Item in Session** column was used as **clustering columns** to sort the data.

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

create_query2 = "CREATE TABLE IF NOT EXISTS song_by_artist "
create_query2 = create_query2 + ('(userId INT, \
                                   sessionId INT, \
                                   itemInSession INT, \
                                   artist TEXT, \
                                   song TEXT, \
                                   firstName TEXT, \
                                   lastName TEXT, \
                                   PRIMARY KEY ((userId, sessionId), itemInSession))')
try:
    session.execute(create_query2)
except Exception as e:
    print(e)                    

#### Insert data into "song_by_artist" table

In [12]:
# Insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `insert_query2` variable
        insert_query2 = "INSERT INTO song_by_artist (userId, \
                                                     sessionId, \
                                                     itemInSession, \
                                                     artist, \
                                                     song, \
                                                     firstName, \
                                                     lastName) "
        insert_query2 = insert_query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insert_query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into "song_by_artist" table

In [13]:
# SELECT statement to verify the data was entered into the table

iteminsession = []
artists = []
songs = []
firstname = []
lastname = []

query2 = "SELECT itemInSession, artist, song, firstName, lastName FROM song_by_artist \
          WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)   

for row in rows:    
    iteminsession.append(row.iteminsession)
    artists.append(row.artist)
    songs.append(row.song)
    firstname.append(row.firstname)
    lastname.append(row.lastname)
    print(row.iteminsession, row.artist, row.song, row.firstname, row.lastname)


0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [14]:
# Create dataframe for Q2
dict2 = {'ItemInSession': iteminsession,
         'artists': artists,
         'songs': songs,
         'firstName': firstname,
         'lastName': lastname}
df2 = pd.DataFrame.from_dict(dict2)
df2

,ItemInSession,artists,songs,firstName,lastName
0,0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,1,Three Drives,Greece 2000,Sylvie,Cruz
2,2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### 3. Create tables for query 3: 
`Table Name: song_by_user
column 1: Song Title
column 2: User ID
column 3: User's first name
column 4: User's last name
PRIMARY KEY (Song Title, User ID )`

**Song Title** and **User ID** are the columns that ares used as **composite PRIMARY KEY** of the table. **Song Title** column was used as **a partiton key** because the queries will filter by this column. **User ID** column was used as **clustering columns** to help make up a unique key.

In [15]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

create_query3 = "CREATE TABLE IF NOT EXISTS song_by_user "
create_query3 = create_query3 + ('(song TEXT, \
                                   userId INT, \
                                   firstName TEXT, \
                                   lastName TEXT, \
                                   PRIMARY KEY (song, userId))')
try:
    session.execute(create_query3)
except Exception as e:
    print(e)                
                    

#### Insert data into "song_by_user" table

In [16]:
# Insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `insert_query3` variable
        insert_query3 = "INSERT INTO song_by_user (song, \
                                                   userId, \
                                                   firstName, \
                                                   lastName) "
        insert_query3 = insert_query3 + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insert_query3, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into "song_by_user" table

In [17]:
# SELECT statement to verify the data was entered into the table

firstname = []
lastname = []

query3 = "SELECT firstName, lastName FROM song_by_user \
          WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:     
    firstname.append(row.firstname)
    lastname.append(row.lastname)
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [18]:
# Create dataframe for Q3
df3 = pd.DataFrame.from_dict({'firstName': firstname,
                              'lastName': lastname})
df3

,firstName,lastName
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### 4. Drop the tables before closing out the sessions

In [19]:
## Drop tables before closing out the sessions

drop_query1 = "DROP TABLE song_by_session"

try:
    session.execute(drop_query1)
except Exception as e:
    print(e)
    
drop_query2 = "DROP TABLE song_by_artist"

try:
    session.execute(drop_query2)
except Exception as e:
    print(e)
    
drop_query3 = "DROP TABLE song_by_user"

try:
    session.execute(drop_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()